# PLSA
_Probabilistic latent semantic analysis_

## Preliminaries
#### Import dependencies

In [ ]:
import sys
import matplotlib.pyplot as plt

#### Set the plotting environment

In [ ]:
%matplotlib notebook

#### Put the actual `plsa` package onto the _python path_

In [ ]:
sys.path.append('..')

#### Import main classes from the `plsa` package

In [ ]:
from plsa import Corpus, Pipeline, Visualize
from plsa.pipeline import DEFAULT_PIPELINE
from plsa.algorithms import PLSA

## Data Sources
As they can be quite large, no actual text corpus is included with the `plsa` package. Two nice examples to play with could be
- [Economic News Article Tone and Relevance](https://www.figure-eight.com/data-for-everyone/)
- [Blog Authorship Corpus](http://u.cs.biu.ac.il/~koppel/BlogCorpus.htm)
We are assuming here that you have downloaded one of them (or both) and placed them under a `data` folder under the
root of your clone of the `PLSA` [GitHub repository](https://github.com/yedivanseven/PLSA).

In [ ]:
csv_file = '../data/Full-Economic-News-DFE-839861.csv'
directory = '../data/blogs'

## Set Up the Corpus
#### Define pre-processing pipeline
Depending on there source, actual, real-world text documents are "dirty", and need to be "cleaned up" through a series of pre-processing steps. The `plsa` submodule `preprocessors` contains several of them
(see the [API documentation](https://probabilistic-latent-semantic-analysis.readthedocs.io/en/latest/plsa.preprocessors.html)). For convenience, they are assembled into a default pipeline that should help you to get
some results out-of-the-box.

In [ ]:
pipeline = Pipeline(*DEFAULT_PIPELINE)
pipeline

#### Load corpus
Execute either this cell ...

In [ ]:
corpus = Corpus.from_xml(directory, pipeline)
corpus

... or that cell:

In [ ]:
corpus = Corpus.from_csv(csv_file, pipeline, delimiter=';')
corpus

## Run PLSA

#### Choose the number of topics

In [ ]:
n_topics = 5

#### Instantiate a PLSA model

In [ ]:
plsa = PLSA(corpus, n_topics, True)
plsa

Notice that we did not do any iterations yet.

#### Fit the PLSA model

In [ ]:
result = plsa.fit()
plsa

Now we indeed did do some iterations.

#### Examine the results
Feel free to explore the attributes of the `result` object. See the [API documentation](https://probabilistic-latent-semantic-analysis.readthedocs.io/en/latest/plsa.algorithms.result.html) for more information.

For example, we could see the relative prevalence of the individual topics we found.

In [ ]:
result.topic

Or, we could predict the topic mixture of an entirely new document.

In [ ]:
new_doc = 'Hello! This is the federal humpty dumpty agency for state funding.'

topic_components, number_of_new_words, new_words = result.predict(new_doc)

print('Relative topic importance in new document:', topic_components)
print('Number of previously unseen words in new document:', number_of_new_words)
print('Previously unseen words in new document:', new_words)

And, of course, we can look at individual topics, that is, how important which word is for which topic. Let's look at the top-10 words of the first topic.

In [ ]:
result.word_given_topic[0][:10]

## Visualize the Results

In [ ]:
visualize = Visualize(result)
visualize

#### Convergence
Since PLSA use an iterative expectation-maximization (EM) style algorithm, let's make sure we have achieved reasonable convergence.

In [ ]:
fig, ax = plt.subplots()
_ = visualize.convergence(ax)
fig.tight_layout()

#### Relative topic importance
How important are the topics we found in the corpus?

In [ ]:
fig, ax = plt.subplots()
_ = visualize.topics(ax)
fig.tight_layout()

#### The topics
The most interesting part is probably the topics themselves, We can visualize them as word clouds.

In [ ]:
fig = plt.figure(figsize=(9.4, 10))
_ = visualize.wordclouds(fig)

#### Relative topic importance in a document
Also interesting is the mixture of topics in each document. Let's look at the first one.

In [ ]:
fig, ax = plt.subplots()
_ = visualize.topics_in_doc(0, ax)
fig.tight_layout()

Let's compare this with what the prediction would look like, pretending that this document wasn't seen before.

In [ ]:
fig, ax = plt.subplots()
_ = visualize.prediction(corpus.raw[0], ax)
fig.tight_layout()

Similar, but not quite the same. This is the very nature of matrix factorization algorithms, to which PLSA can be seen to belong. We try to approxmiate the original counts of each word in each document with a lower-dimensional representation of the data. That's why the topic composition get's somewhat "blurred".

#### Prediction for a new document
We can also visualize the predicited topic composition for a new document.

In [ ]:
new_doc = 'Hello! This is the federal humpty dumpty agency for state funding.'

fig, ax = plt.subplots()
_ = visualize.prediction(new_doc, ax)
fig.tight_layout()